In [28]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PyCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 30242
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Mary Cigliola - CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'height': '150px',
                'width': 'auto',
                'display': 'block',
                'margin-left': 'auto',
                'margin-right': 'auto'
            }),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DR'},
                {'label': 'Reset', 'value': 'reset'}], 
            inline = True, id = 'rescue-filter')
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# Display the data in the table, including applying selected filters
    
@app.callback(
              [Output('datatable-id','data'), 
              Output('datatable-id', 'columns')],
              [Input('rescue-filter', 'value')])
def update_dashboard(rescue_type):
    # Water Rescue
    if rescue_type == 'WR': 
        result = shelter.read({"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, 
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {'$gte': 26.0, '$lte': 156.0}})
        df=pd.DataFrame.from_records(result)
        
    # Mountain/Wilderness Rescue
    elif rescue_type == 'MWR': 
        result = shelter.read({"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                          "Siberian Husky", "Rottweiler"]}, 
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {'$gte': 26.0, '$lte': 156.0}})
        df=pd.DataFrame.from_records(result)
        
    #Disaster Rescue/Individual Tracking
    elif rescue_type == 'DR': 
        result = shelter.read({"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound",
                          "Rottweiler"]}, 
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {'$gte': 20.0, '$lte': 300.0}})
        df=pd.DataFrame.from_records(result)
        
    # No filters applied (filters reset)
    else:
        result = shelter.read({})
        df = pd.DataFrame.from_records(result)
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True, errors='ignore')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data, columns)


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(Output('graph-id', "children"),
           [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    dff = pd.DataFrame(viewData)  
    
    # Count the number of animals in each breed type
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    breed_counts = breed_counts.sort_values(by='count', ascending=False)
    # If more than 6 breeds are represented, only display the top 6
    # Remaining breeds are grouped as 'other'
    if len(breed_counts) > 6:
        top_breeds = breed_counts[:6]
        other_count = breed_counts[6:]['count'].sum()
        other_breeds = pd.DataFrame([{'breed': 'Other', 'count': other_count}])
        breed_counts = pd.concat([top_breeds, other_breeds], ignore_index=True)
    else:
        breed_counts = breed_counts
    
    chart = px.pie(breed_counts, values='count', names='breed', title='Percentage of Breeds')
    return [dcc.Graph(figure=chart)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'),
            [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(Output('map-id', "children"),
            [Input('datatable-id', "derived_virtual_data"),
             Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:26696/
                         _id  rec_num age_upon_outcome animal_id animal_type  \
0   65ff710b8dc95b7efffe3270       36         6 months   A706953         Dog   
1   65ff710b8dc95b7efffe3528      732          2 years   A749782         Dog   
2   65ff710b8dc95b7efffe36b9     1121           1 year   A757158         Dog   
3   65ff710b8dc95b7efffe38a8     1628         9 months   A740471         Dog   
4   65ff710b8dc95b7efffe3924     1757         7 months   A742767         Dog   
5   65ff710b8dc95b7efffe3a10     1988           1 year   A762781         Dog   
6   65ff710b8dc95b7efffe3a51     2041          2 years   A702745         Dog   
7   65ff710b8dc95b7efffe3af6     2225          2 years   A757341         Dog   
8   65ff710b8dc95b7efffe3f52     3319         9 months   A687748         Dog   
9   65ff710b8dc95b7efffe42ca     4222           1 year   A735551         Dog   
10  65ff710b8dc95b7efffe4380     4391          2 years   A739950         Dog